# common

In [ ]:
//// test

open testing

## types

In [ ]:
inl types () =
    env.types ()
    rust.types ()
    sm'.types ()

## common

### (:>)

In [ ]:
prototype (~:>) r : forall t. t -> r

### to_any

In [ ]:
inl to_any forall t. (obj : t) : any =
    $'!obj '

instance (~:>) any = to_any

### (||>)

In [ ]:
//// test

(3i32, 2i32)
||> fun a b => a - b
|> _assert_eq 1

assert_eq / actual: 1 / expected: 1


### flip

In [ ]:
inl flip fn a b =
    fn b a

In [ ]:
//// test

(1i32, 2i32)
||> flip pair
|> _assert_eq (2, 1)

assert_eq / actual: struct (2, 1) / expected: struct (2, 1)


### join_body

In [ ]:
inl join_body body acc x =
    if var_is x |> not
    then body acc x
    else
        inl acc = dyn acc
        join body acc x

In [ ]:
//// test

inl rec fold_list f s = function
    | Cons (x, x') => fold_list f (f s x) x'
    | Nil => s

In [ ]:
//// test

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (+) 0
|> _assert_eq 15

assert_eq / actual: 15 / expected: 15


In [ ]:
//// test

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (join_body (+)) 0
|> _assert_eq 15

assert_eq / actual: 15 / expected: 15


### join_body_unit

In [ ]:
inl join_body_unit body d x =
    if var_is d |> not
    then body x
    else
        inl x = dyn x
        join body x

In [ ]:
//// test

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (fun acc n => join_body_unit ((+) acc) n n) 0
|> _assert_eq 15

assert_eq / actual: 15 / expected: 15


### retry_fn'

In [ ]:
inl retry_fn' retries fn =
    let rec loop retry =
        inl is_error, result =
            match fn () with
            | Ok x => false, x
            | Error x => true, x
        if not is_error || retry >= retries
        then result
        else
            trace Debug
                fun () => $'"common.retry_fn\' / loop"'
                fun () => $'$"is_error: {!is_error} / retry: {!retry}/{!retries} / result: %A{!result} / {!_locals ()}"'
            loop (retry + 1)
    loop 1

## fsharp

### nameof

In [ ]:
inl nameof x : string =
    $'nameof !x '

### upcast

In [ ]:
inl upcast forall t u. (x : t) : u =
    $'!x :> `u '

### downcast

In [ ]:
inl downcast forall t u. (x : t) : u =
    $'!x :?> `u '

### random

In [ ]:
nominal random = $'System.Random'

inl random () : random =
    $'`random ' ()

### random_next

In [ ]:
inl random_next (min : i32) (max : i32) (random : random) : i32 =
    $'!random.Next (!min, !max)'

### disposable

In [ ]:
nominal disposable t = $'System.IDisposable'

### dispose

In [ ]:
inl dispose (disposable : disposable _) : () =
    disposable |> $'_.Dispose()'

### return

In [ ]:
inl return forall t. (x : t) : () =
    $'return !x '

inl return' forall t. (x : t) : t =
    $'return !x '

### retry_fn

In [ ]:
inl retry_fn forall t. retries (fn : () -> t) : option t =
    let rec loop retry =
        try
            fun () =>
                if retry < retries
                then fn () |> Some
                else None
            fun ex =>
                trace Warning
                    fun () => "common.retry_fn"
                    fun () => $'$"retry: {!retry} / ex: %A{!ex} / {!_locals ()}"'
                None
        |> function
            | Some x => x
            | None => loop (retry + 1)
    loop 0

In [ ]:
//// test

inl retry_fn_test = mut 0i32
fun () =>
    retry_fn_test <- *retry_fn_test + 1
    *retry_fn_test
|> retry_fn 3i32
|> _assert_eq (Some 1i32)

assert_eq / actual: US0_0 1 / expected: US0_0 1


In [ ]:
//// test
///! rust

types ()
inl retry_fn_test = mut 0i32
fun () =>
    retry_fn_test <- *retry_fn_test + 1
    *retry_fn_test
|> retry_fn 3i32
|> _assert_eq' (Some 1i32)

assert_eq' / actual: US0_0(1) / expected: US0_0(1)


In [ ]:
//// test

inl retry_fn_test = mut 0i32
fun () =>
    if *retry_fn_test >= 2
    then *retry_fn_test
    else
        retry_fn_test <- *retry_fn_test + 1
        failwith "test"
|> retry_fn 3i32
|> _assert_eq (Some 2i32)

00:00:00 warning #1 common.retry_fn / retry: 0 / ex: System.Exception: test
   at FSI_0028.closure0(Mut0 v0, Int32 v1, Unit unitVar2)
   at FSI_0028.method1(Mut0 v0, Int32 v1)
00:00:00 warning #2 common.retry_fn / retry: 1 / ex: System.Exception: test
   at FSI_0028.closure0(Mut0 v0, Int32 v1, Unit unitVar2)
   at FSI_0028.method1(Mut0 v0, Int32 v1)
assert_eq / actual: US0_0 2 / expected: US0_0 2


In [ ]:
//// test
///! rust

types ()
inl retry_fn_test = mut 0i32
fun () =>
    if *retry_fn_test >= 2
    then *retry_fn_test
    else
        retry_fn_test <- *retry_fn_test + 1
        failwith "test"
|> retry_fn 3i32
|> _assert_eq' (Some 2i32)

00:00:00 warning #1 common.retry_fn / retry: 0 / ex: Exception { message: "test" }
00:00:00 warning #2 common.retry_fn / retry: 1 / ex: Exception { message: "test" }
assert_eq' / actual: US0_0(2) / expected: US0_0(2)


## common

### random'

In [ ]:
inl random' forall t. (min : t) (max : t) : t =
    run_target function
        | Rust (Contract) => fun () =>
            failwith "common.random' / target=Rust(Contract)"
        | Rust _ => fun () =>
            open rust_operators
            !\\((min, max), $'"rand::Rng::gen_range(&mut rand::thread_rng(), $0..$1)"')
        | _ => fun () =>
            random () |> random_next (i32 min) (i32 max) |> convert

### new_disposable

In [ ]:
inl new_disposable (fn : () -> ()) : disposable _ =
    run_target function
        | Rust _ => fun () =>
            global "type Disposable (f : unit -> unit) = interface System.IDisposable with member _.Dispose () = f ()"
            inl fn = join fn
            $'new Disposable (fun () -> Fable.Core.RustInterop.emitRustExpr !fn "$0()" )'
        | Fsharp _ => fun () =>
            inl fn = join fn
            $'{ new System.IDisposable with member _.Dispose () = !fn () }'
        | _ => fun () => null ()

In [ ]:
//// test

inl new_disposable_test = mut 0i32
new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
|> fun x => x : disposable ()
|> dispose
*new_disposable_test |> _assert_eq 1

assert_eq / actual: 1 / expected: 1


In [ ]:
//// test
///! rust

inl new_disposable_test = mut 0i32
new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
|> fun x => x : disposable ()
|> dispose
*new_disposable_test |> _assert_eq 1

assert_eq / actual: 1 / expected: 1


In [ ]:
//// test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> use
    |> ignore
    |> return
|> async.new_task
|> async.await_task
|> async.run_synchronously
*new_disposable_test |> _assert_eq 1

assert_eq / actual: 1 / expected: 1


In [ ]:
//// test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> use
    |> ignore
    |> return
|> async.new_async
|> async.run_synchronously
*new_disposable_test |> _assert_eq 1

assert_eq / actual: 1 / expected: 1


In [ ]:
//// test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> ignore
    |> return
|> async.new_async
|> async.run_synchronously
*new_disposable_test |> _assert_eq 0

assert_eq / actual: 0 / expected: 0


## main

In [ ]:
inl main () =
    types ()
    init_trace_state None
    inl new_disposable x : _ () = new_disposable x
    $'let new_disposable x = !new_disposable x' : ()

    inl retry_fn (r : i32) (x : () -> _) : optionm'.option' () = retry_fn r x |> optionm'.box
    $'let retry_fn x = !retry_fn x' : ()
    inl memoize (fn : () -> ()) : () -> () = memoize fn
    $'let memoize x = !memoize x' : ()